In [1]:
import os

print(os.path.abspath(os.curdir))

d:\Data Kuliah\Semester 8\IEEE - TIFS\dataset\processed\airdata\notebooks


In [2]:
import os
os.chdir("..")

In [3]:
print(os.path.abspath(os.curdir))

d:\Data Kuliah\Semester 8\IEEE - TIFS\dataset\processed\airdata


In [ ]:
import pandas as pd
from openpyxl import load_workbook

def apply_corrections():
    # Load the correction log
    correction_log = pd.read_excel('correction_log_step_3.xlsx')
    
    # Load the previous corrected data (assuming it's in corrected_data_step_2.xlsx)
    try:
        corrected_data = pd.read_excel('corrected_data_step_2.xlsx')
    except FileNotFoundError:
        print("Error: corrected_data_step_2.xlsx not found. Please ensure it's in the same directory.")
        return
    
    # Find the maximum MessageID in the corrected data to start assigning new IDs
    max_id = corrected_data['MessageID'].max()
    new_id = max_id + 1
    
    # Process 'drop' corrections first (remove these messages)
    drop_ids = correction_log[correction_log['Correction Type'] == 'drop']['MessageID'].unique()
    corrected_data = corrected_data[~corrected_data['MessageID'].isin(drop_ids)]
    
    # Process 'over-generalized' corrections
    over_generalized = correction_log[correction_log['Correction Type'] == 'over-generalized']
    
    # We'll collect all new rows to add
    new_rows = []
    
    for _, row in over_generalized.iterrows():
        original_id = row['MessageID']
        corrected_message = row['Corrected Message']
        justification = row['Justification']
        
        # Find the original message in the corrected data
        original_message = corrected_data[corrected_data['MessageID'] == original_id]
        
        if not original_message.empty:
            # Create new rows for each variant
            new_row = original_message.iloc[0].copy()
            new_row['MessageID'] = new_id
            new_row['Message'] = corrected_message
            # new_row['Justification'] = justification
            new_rows.append(new_row)
            new_id += 1
    
    # Add the new rows to the corrected data
    if new_rows:
        new_rows_df = pd.DataFrame(new_rows)
        corrected_data = pd.concat([corrected_data, new_rows_df], ignore_index=True)
    
    # Remove the original messages that were over-generalized (since we've created variants)
    over_generalized_ids = over_generalized['MessageID'].unique()
    corrected_data = corrected_data[~corrected_data['MessageID'].isin(over_generalized_ids)]
    
    # Save the final corrected data
    corrected_data.to_excel('corrected_data_step_3.xlsx', index=False)
    print("Corrections applied successfully. Results saved to corrected_data_step_3.xlsx")

if __name__ == "__main__":
    apply_corrections()

Corrections applied successfully. Results saved to corrected_data_step_3.xlsx
